# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [278]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [279]:
total_schools = school_data_complete['school_name'].nunique()
total_students = school_data_complete["Student ID"].count()
total_budget = school_data_complete[["school_name","budget"]].drop_duplicates()["budget"].sum()
avg_math_score = school_data_complete["math_score"].mean()
avg_reading_score = school_data_complete["reading_score"].mean()
pct_pass_math = school_data_complete.loc[school_data_complete["math_score"] >= 70,:]["math_score"].count() / school_data_complete["math_score"].count()
pct_pass_reading = school_data_complete.loc[school_data_complete["reading_score"] >= 70,:]["reading_score"].count() / school_data_complete["reading_score"].count()
overall_pass_rate = (avg_math_score+avg_reading_score)/2

results = pd.DataFrame(np.column_stack([total_schools, total_students, total_budget, avg_math_score, avg_reading_score, pct_pass_math, pct_pass_reading, overall_pass_rate]), 
                               columns=['Total Schools','Total Students','Total Budget','Average Math Score','Average Reading Score','% Passing Math','% Passing Reading','% Overall Passing Rate'])

results.head()

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15.0,39170.0,24649428.0,78.985371,81.87784,0.749809,0.858055,80.431606


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [280]:
type_school = school_data_complete[["school_name","type"]].drop_duplicates().set_index('school_name').rename(columns = {
    'type': "School Type" })
total_student = pd.DataFrame(school_data_complete.groupby('school_name')['student_name'].count()).rename(columns = {
    'student_name': "Total Students" })
total_school_budget = school_data_complete[["school_name","budget"]].drop_duplicates().set_index('school_name').rename(columns = {
    'budget': "Total School Budget" })
avg_math_score = pd.DataFrame(school_data_complete.groupby('school_name')['math_score'].mean()).rename(columns = {
    'math_score': "Average Math Score" })
avg_reading_score = pd.DataFrame(school_data_complete.groupby('school_name')['reading_score'].mean()).rename(columns = {
    'reading_score': "Average Reading Score" })
per_pass_math = pd.DataFrame(school_data_complete.loc[school_data_complete["math_score"] >= 70,
                                      :].groupby('school_name')["math_score"].count() / 
             school_data_complete.groupby('school_name')["student_name"].count()*100).rename(columns = {0: "% Passing Math" })

per_pass_reading = pd.DataFrame(school_data_complete.loc[school_data_complete["reading_score"] >= 70,
                                      :].groupby('school_name')["reading_score"].count() / 
             school_data_complete.groupby('school_name')["student_name"].count()*100).rename(columns = {0: "% Passing Reading" })

school_summary = type_school.join([total_student,total_school_budget,avg_math_score,avg_reading_score,per_pass_math,per_pass_reading])
school_summary['Per Student Budget'] = school_summary["Total School Budget"] / school_summary["Total Students"]
school_summary['% Overall Passing Rate'] = (school_summary["% Passing Math"] + school_summary["% Passing Reading"])/2
school_summary = school_summary[['School Type', 'Total Students', 'Total School Budget', 'Per Student Budget',
                                'Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading',
                                '% Overall Passing Rate']].sort_values(by ='% Overall Passing Rate', ascending=False).head()

del school_summary.index.name
school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [281]:
type_school = school_data_complete[["school_name","type"]].drop_duplicates().set_index('school_name').rename(columns = {
    'type': "School Type" })
total_student = pd.DataFrame(school_data_complete.groupby('school_name')['student_name'].count()).rename(columns = {
    'student_name': "Total Students" })
total_school_budget = school_data_complete[["school_name","budget"]].drop_duplicates().set_index('school_name').rename(columns = {
    'budget': "Total School Budget" })
avg_math_score = pd.DataFrame(school_data_complete.groupby('school_name')['math_score'].mean()).rename(columns = {
    'math_score': "Average Math Score" })
avg_reading_score = pd.DataFrame(school_data_complete.groupby('school_name')['reading_score'].mean()).rename(columns = {
    'reading_score': "Average Reading Score" })
per_pass_math = pd.DataFrame(school_data_complete.loc[school_data_complete["math_score"] >= 70,
                                      :].groupby('school_name')["math_score"].count() / 
             school_data_complete.groupby('school_name')["student_name"].count()*100).rename(columns = {0: "% Passing Math" })

per_pass_reading = pd.DataFrame(school_data_complete.loc[school_data_complete["reading_score"] >= 70,
                                      :].groupby('school_name')["reading_score"].count() / 
             school_data_complete.groupby('school_name')["student_name"].count()*100).rename(columns = {0: "% Passing Reading" })

school_summary = type_school.join([total_student,total_school_budget,avg_math_score,avg_reading_score,per_pass_math,per_pass_reading])
school_summary['Per Student Budget'] = school_summary["Total School Budget"] / school_summary["Total Students"]
school_summary['% Overall Passing Rate'] = (school_summary["% Passing Math"] + school_summary["% Passing Reading"])/2
school_summary = school_summary[['School Type', 'Total Students', 'Total School Budget', 'Per Student Budget',
                                'Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading',
                                '% Overall Passing Rate']].sort_values(by ='% Overall Passing Rate').head()

del school_summary.index.name
school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [282]:
avg_math_score_9thgrade = pd.DataFrame(school_data_complete.loc[school_data_complete["grade"] == "9th",
                                      :].groupby('school_name')["math_score"].mean()).rename(columns = {
    'math_score': "9th" })
avg_math_score_10thgrade = pd.DataFrame(school_data_complete.loc[school_data_complete["grade"] == "10th",
                                      :].groupby('school_name')["math_score"].mean()).rename(columns = {
    'math_score': "10th" })
avg_math_score_11thgrade = pd.DataFrame(school_data_complete.loc[school_data_complete["grade"] == "11th",
                                      :].groupby('school_name')["math_score"].mean()).rename(columns = {
    'math_score': "11th" })
avg_math_score_12thgrade = pd.DataFrame(school_data_complete.loc[school_data_complete["grade"] == "12th",
                                      :].groupby('school_name')["math_score"].mean()).rename(columns = {
    'math_score': "12th" })

math_scores_by_grade = avg_math_score_9thgrade.join([avg_math_score_10thgrade,avg_math_score_11thgrade,avg_math_score_12thgrade])
del math_scores_by_grade.index.name
math_scores_by_grade

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [283]:
school_data_complete.head()

avg_reading_score_9thgrade = pd.DataFrame(school_data_complete.loc[school_data_complete["grade"] == "9th",
                                      :].groupby('school_name')["reading_score"].mean()).rename(columns = {
    'reading_score': "9th" })
avg_reading_score_10thgrade = pd.DataFrame(school_data_complete.loc[school_data_complete["grade"] == "10th",
                                      :].groupby('school_name')["reading_score"].mean()).rename(columns = {
    'reading_score': "10th" })
avg_reading_score_11thgrade = pd.DataFrame(school_data_complete.loc[school_data_complete["grade"] == "11th",
                                      :].groupby('school_name')["reading_score"].mean()).rename(columns = {
    'reading_score': "11th" })
avg_reading_score_12thgrade = pd.DataFrame(school_data_complete.loc[school_data_complete["grade"] == "12th",
                                      :].groupby('school_name')["reading_score"].mean()).rename(columns = {
    'reading_score': "12th" })

reading_scores_by_grade = avg_math_score_9thgrade.join([avg_reading_score_10thgrade,avg_reading_score_11thgrade,avg_reading_score_12thgrade])
reading_scores_by_grade

,9th,10th,11th,12th
Bailey High School,77.083676,80.907183,80.945643,80.912451
Cabrera High School,83.094697,84.253219,83.788382,84.287958
Figueroa High School,76.403037,81.408912,80.640339,81.384863
Ford High School,77.361345,81.262712,80.403642,80.662338
Griffin High School,82.044010,83.706897,84.288089,84.013699
Hernandez High School,77.438495,80.660147,81.396140,80.857143
Holden High School,83.787402,83.324561,83.815534,84.698795
Huang High School,77.027251,81.512386,81.417476,80.305983
Johnson High School,77.187857,80.773431,80.616027,81.227564
Pena High School,83.625455,83.612000,84.335938,84.591160


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [284]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [285]:
school_data_complete['Spending Ranges (Per Student)'] = pd.cut(school_data_complete['budget']/school_data_complete['size'], bins, labels = group_names)

avg_math_score = pd.DataFrame(school_data_complete.groupby('Spending Ranges (Per Student)')['math_score'].mean()).rename(columns = {
    'math_score': "Average Math Score" })
avg_reading_score = pd.DataFrame(school_data_complete.groupby('Spending Ranges (Per Student)')['reading_score'].mean()).rename(columns = {
    'reading_score': "Average Reading Score" })
per_pass_math = pd.DataFrame(school_data_complete.loc[school_data_complete["math_score"] >= 70,
                                      :].groupby('Spending Ranges (Per Student)')["math_score"].count() / 
             school_data_complete.groupby('Spending Ranges (Per Student)')["student_name"].count()*100).rename(columns = {
    0: "% Passing Math" })
per_pass_reading = pd.DataFrame(school_data_complete.loc[school_data_complete["reading_score"] >= 70,
                                      :].groupby('Spending Ranges (Per Student)')["reading_score"].count() / 
             school_data_complete.groupby('Spending Ranges (Per Student)')["student_name"].count()*100).rename(columns = {
    0: "% Passing Reading" })

score_by_school_spreading = avg_math_score.join([avg_reading_score,per_pass_math,per_pass_reading])
score_by_school_spreading['% Overall Passing Rate'] = (score_by_school_spreading["% Passing Math"] + score_by_school_spreading["% Passing Reading"])/2
score_by_school_spreading

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.363065,83.964039,93.702889,96.686558,95.194724
$585-615,83.529196,83.838414,94.124128,95.886889,95.005509
$615-645,78.061635,81.434088,71.400428,83.614770,77.507599
$645-675,77.049297,81.005604,66.230813,81.109397,73.670105


## Scores by School Size

* Perform the same operations as above, based on school size.

In [286]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [287]:
school_data_complete['School Size'] = pd.cut(school_data_complete['size'], size_bins, labels = group_names)

avg_math_score = pd.DataFrame(school_data_complete.groupby('School Size')['math_score'].mean()).rename(columns = {
    'math_score': "Average Math Score" })
avg_reading_score = pd.DataFrame(school_data_complete.groupby('School Size')['reading_score'].mean()).rename(columns = {
    'reading_score': "Average Reading Score" })
per_pass_math = pd.DataFrame(school_data_complete.loc[school_data_complete["math_score"] >= 70,
                                      :].groupby('School Size')["math_score"].count() / 
             school_data_complete.groupby('School Size')["student_name"].count()*100).rename(columns = {
    0: "% Passing Math" })
per_pass_reading = pd.DataFrame(school_data_complete.loc[school_data_complete["reading_score"] >= 70,
                                      :].groupby('School Size')["reading_score"].count() / 
             school_data_complete.groupby('School Size')["student_name"].count()*100).rename(columns = {
    0: "% Passing Reading" })

score_by_school_size = avg_math_score.join([avg_reading_score,per_pass_math,per_pass_reading])
score_by_school_size['% Overall Passing Rate'] = (score_by_school_size["% Passing Math"] + score_by_school_size["% Passing Reading"])/2
score_by_school_size

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.828654,83.974082,93.952484,96.040317,94.996400
Medium (1000-2000),83.372682,83.867989,93.616522,96.773058,95.194790
Large (2000-5000),77.477597,81.198674,68.652380,82.125158,75.388769


## Scores by School Type

* Perform the same operations as above, based on school type.

In [288]:
school_data_complete["Type"] = school_data_complete["type"].replace({"Charter": 1, "District":2})

bins = [0, 1, 2]
group_names = ["Charter", "District"]

school_data_complete['School Type'] = pd.cut(school_data_complete['Type'], bins, labels = group_names)

avg_math_score = pd.DataFrame(school_data_complete.groupby('School Type')['math_score'].mean()).rename(columns = {
    'math_score': "Average Math Score" })
avg_reading_score = pd.DataFrame(school_data_complete.groupby('School Type')['reading_score'].mean()).rename(columns = {
    'reading_score': "Average Reading Score" })
per_pass_math = pd.DataFrame(school_data_complete.loc[school_data_complete["math_score"] >= 70,
                                      :].groupby('School Type')["math_score"].count() / 
             school_data_complete.groupby('School Type')["student_name"].count()*100).rename(columns = {
    0: "% Passing Math" })
per_pass_reading = pd.DataFrame(school_data_complete.loc[school_data_complete["reading_score"] >= 70,
                                      :].groupby('School Type')["reading_score"].count() / 
             school_data_complete.groupby('School Type')["student_name"].count()*100).rename(columns = {
    0: "% Passing Reading" })

score_by_school_type = avg_math_score.join([avg_reading_score,per_pass_math,per_pass_reading])
score_by_school_type['% Overall Passing Rate'] = (score_by_school_type["% Passing Math"] + score_by_school_type["% Passing Reading"])/2
score_by_school_type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.406183,83.902821,93.701821,96.645891,95.173856
District,76.987026,80.962485,66.518387,80.905249,73.711818
